In [1]:
!pip install tensorflow --use-feature=2020-resolver

In [2]:
import time
start_time = time.perf_counter()

In [14]:
import ray
ray.init(address="auto")


2022-07-27 04:16:52,970	INFO worker.py:946 -- Connecting to existing Ray cluster at address: 172.24.31.101:6379


RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

In [17]:
ray.nodes()

[{'NodeID': '5671f6d6e692cc0aa29ef688b5494d2f6cb7b5d56cdebbefaaba42ac',
  'Alive': True,
  'NodeManagerAddress': '172.24.31.101',
  'NodeManagerHostname': 'mgmt01',
  'NodeManagerPort': 39889,
  'ObjectManagerPort': 37609,
  'ObjectStoreSocketName': '/tmp/ray/session_2022-06-02_02-37-25_924604_37/sockets/plasma_store',
  'RayletSocketName': '/tmp/ray/session_2022-06-02_02-37-25_924604_37/sockets/raylet',
  'MetricsExportPort': 56458,
  'alive': True,
  'Resources': {'object_store_memory': 4895835340.0,
   'CPU': 4.0,
   'memory': 9791670683.0,
   'node:172.24.31.101': 1.0}},
 {'NodeID': '59496ae8c13baa067dd4e28bc95cdd1fe531e2314f7f0c5231ea94d2',
  'Alive': True,
  'NodeManagerAddress': '172.24.31.103',
  'NodeManagerHostname': 'gpu02',
  'NodeManagerPort': 36761,
  'ObjectManagerPort': 41267,
  'ObjectStoreSocketName': '/tmp/ray/session_2022-06-02_02-37-25_924604_37/sockets/plasma_store',
  'RayletSocketName': '/tmp/ray/session_2022-06-02_02-37-25_924604_37/sockets/raylet',
  'MetricsE

In [4]:
# =============================
# Validating Tensorflow version
# =============================
tic = time.perf_counter()
print("* Stage 1: Initializing TensorFlow")
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

toc = time.perf_counter()
print("Stage 1 completed in", round(toc - tic, 2), "seconds")

* Stage 1: Initializing TensorFlow
Stage 1 completed in 3.21 seconds


In [5]:
# ==========================
# Importing common libraries
# ==========================
tic = time.perf_counter()
print("* Stage 2: Importing common libraries")
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # suppress AVX2 FMA not supported error
import urllib
import tarfile
import numpy as np
toc = time.perf_counter()
print("Stage 2 completed in", round(toc - tic, 2), "seconds")

* Stage 2: Importing common libraries
Stage 2 completed in 0.19 seconds


In [6]:
# ============
# Helper Codes
# ============
# Font to draw text on image
FONT_NAME = 'Chalkduster.ttf'

# Bounding box colors
COLORS = ['Green',
          'Red', 'Pink',
          'Olive', 'Brown', 'Gray',
          'Cyan', 'Orange']

class ObjectResult:
    """
    Represents a detection result, containing the object label,
    score confidence, and bounding box coordinates.
    """

    def __init__(self, label, score, box):
        self.label = label
        self.score = score
        self.box = box
    
    def __repr__(self):
        return '{0} ({1}%)'.format(self.label, int(100 * self.score))


N_CHANNELS = 3

def load_image_into_numpy_array(image):
    """
    Converts a PIL image into a numpy array (height x width x channels).
    :param image: PIL image
    :return: numpy array
    """
    (width, height) = image.size
    return np.array(image.getdata()) \
        .reshape((height, width, N_CHANNELS)).astype(np.uint8)

def process_output(classes, scores, boxes, category_index):
    """
    Processes classes, scores, and boxes, gathering in a list of ObjectResult.
    :param classes: list of class id
    :param scores: list of scores
    :param boxes: list of boxes
    :param category_index: label dictionary
    :return: list of ObjectResult
    """
    results = []

    for clazz, score, box in zip(classes, scores, boxes):
        if score > 0.0:
            label = category_index[clazz][NAME_KEY]
            obj_result = ObjectResult(label, score, box)
            results.append(obj_result)
    
    return results

import random
import PIL.Image as Image

def draw_labeled_boxes(image_np, results, min_score=.4):
    """
    Draws labeled boxes according to results on the given image.
    :param image_np: numpy array image
    :param results: list of ObjectResult
    :param min_score: optional min score threshold, default is 40%
    :return: numpy array image with labeled boxes drawn
    """
    results.sort(key=lambda x: x.score, reverse=False)
    image_np_copy = image_np.copy()
    for r in results:
        if r.score >= min_score:
            color_idx = random.randint(0, len(COLORS) - 1)
            color = COLORS[color_idx]

            image_pil = Image.fromarray(np.uint8(image_np_copy)).convert('RGB')
            draw_bounding_box_on_image(image_pil, r.box, color, str(r))
            np.copyto(image_np_copy, np.array(image_pil))

    return image_np_copy

import PIL.ImageFont as ImageFont

def get_suitable_font_for_text(text, img_width, font_name, img_fraction=0.12):
    """
    Calculates a suitable font for the image given the text and fraction.
    :param text: text that will be drawn
    :param img_width: width of the image
    :param font_name: name of the font
    :param img_fraction: optional desired image fraction allowed for the text 
    :return: suitable font
    """
    fontsize = 1
    font = ImageFont.truetype(FONT_NAME, fontsize)
    while font.getsize(text)[0] < img_fraction*img_width:
        fontsize += 1
        font = ImageFont.truetype(font_name, fontsize)
    return font

import PIL.ImageDraw as ImageDraw

TEXT_COLOR = 'Black'

def draw_bounding_box_on_image(image, box, color, box_label):
    """
    Draws the box and label on the given image.
    :param image: PIL image
    :param box: numpy array containing the bounding box information
                [top, left, bottom, right]
    :param color: bounding box color
    :param box_label: bounding box label
    """
    im_width, im_height = image.size
    top, left, bottom, right = box

    # Normalize coordinates
    left = left * im_width
    right = right * im_width
    top = top * im_height
    bottom = bottom * im_height

    # Draw the detected bounding box
    line_width = int(max(im_width, im_height) * 0.005)
    draw = ImageDraw.Draw(image)
    draw.rectangle(((left, top), (right, bottom)),
                   width=line_width,
                   outline=color)

    # Get a suitable font (in terms of size with respect to the image)
    font = get_suitable_font_for_text(box_label, im_width, FONT_NAME)
    text_width, text_height = font.getsize(box_label)

    # Draw the box label rectangle
    text_bottom = top + text_height
    text_rect = ((left, top),
                 (left + text_width + 2 * line_width,
                  text_bottom + 2 * line_width))
    draw.rectangle(text_rect, fill=color)

    # Draw the box label text 
    # right below the upper-left horizontal line of the bounding box
    text_position = (left + line_width, top + line_width)
    draw.text(text_position, box_label, fill=TEXT_COLOR, font=font)

# Input tensor
IMAGE_TENSOR_KEY = 'image_tensor'

# Output tensors
DETECTION_BOXES_KEY = 'detection_boxes'
DETECTION_SCORES_KEY = 'detection_scores'
DETECTION_CLASSES_KEY = 'detection_classes'

TENSOR_SUFFIX = ':0'

def run_inference(graph, image_np):
    """
    Runs the inference on the given image.
    :param graph: tensorflow graph
    :param image_np: numpy image
    :return: dictionary with detected classes 
             and their corresponding scores and boxes
    """
    output_tensor_dict = {
        DETECTION_BOXES_KEY: DETECTION_BOXES_KEY + TENSOR_SUFFIX,
        DETECTION_SCORES_KEY: DETECTION_SCORES_KEY + TENSOR_SUFFIX,
        DETECTION_CLASSES_KEY: DETECTION_CLASSES_KEY + TENSOR_SUFFIX
    }

    with graph.as_default():
        with tf.compat.v1.Session() as sess:
            input_tensor = tf.compat.v1.get_default_graph()\
                .get_tensor_by_name(IMAGE_TENSOR_KEY + TENSOR_SUFFIX)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            input_tensor_dict = {input_tensor: image_np_expanded}
            output_dict = sess.run(output_tensor_dict,
                                   feed_dict=input_tensor_dict)

            return {
                DETECTION_BOXES_KEY: 
                    output_dict[DETECTION_BOXES_KEY][0],
                DETECTION_SCORES_KEY: 
                    output_dict[DETECTION_SCORES_KEY][0],
                DETECTION_CLASSES_KEY: 
                    output_dict[DETECTION_CLASSES_KEY][0].astype(np.int64)
            }

In [7]:
# ======================
# Getting the model file
# ======================
tic = time.perf_counter()
print("* Stage 3: Getting the model file")
MODEL_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
MODEL_FILE = MODEL_NAME + '.tar.gz'

if os.path.exists(MODEL_FILE) is False:
    opener = urllib.request.URLopener()
    opener.retrieve(MODEL_DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
toc = time.perf_counter()
print("Stage 3 completed in", round(toc - tic, 2), "seconds")

* Stage 3: Getting the model file
Stage 3 completed in 0.0 seconds


In [8]:
# ================================
# Getting the box description file
# ================================
tic = time.perf_counter()
print("* Stage 4: Getting the box description file")
BOX_DESCRIPTIONS_FILE = 'class-descriptions-boxable.csv'
OID_DOWNLOAD_BASE = 'https://storage.googleapis.com/openimages/2018_04/'

if os.path.exists(BOX_DESCRIPTIONS_FILE) is False:
    opener = urllib.request.URLopener()
    opener.retrieve(OID_DOWNLOAD_BASE + BOX_DESCRIPTIONS_FILE, BOX_DESCRIPTIONS_FILE)
toc = time.perf_counter()
print("Stage 4 completed in", round(toc - tic, 2), "seconds")

* Stage 4: Getting the box description file
Stage 4 completed in 0.0 seconds


In [9]:
# ===================
# Getting test images
# ===================
tic = time.perf_counter()
print("* Stage 5: Getting test images")
import glob

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

TEST_IMAGES = []
FOLDER_IMAGES = 'images/'
IMG_FILTER = '*.jpg'
if os.path.exists(FOLDER_IMAGES):
    TEST_IMAGES = glob.glob(FOLDER_IMAGES + IMG_FILTER)
else:
	print("Folder not found")
toc = time.perf_counter()
print("Stage 5 completed in", round(toc - tic, 2), "seconds")

* Stage 5: Getting test images
Stage 5 completed in 0.0 seconds


In [10]:
# ==========================
# Extracting the model files
# ==========================
tic = time.perf_counter()
print("* Stage 6: Extracting the model files")
FROZEN_GRAPH_FILE = 'frozen_inference_graph.pb'
PATH_TO_FROZEN_GRAPH = os.path.join(MODEL_NAME, FROZEN_GRAPH_FILE)

if os.path.exists(MODEL_NAME) is False:
    tar_file = tarfile.open(MODEL_FILE)
    for file in tar_file.getmembers():
        filename = os.path.basename(file.name)
        if FROZEN_GRAPH_FILE in filename:
            tar_file.extract(file, os.getcwd())
toc = time.perf_counter()
print("Stage 6 completed in", round(toc - tic, 2), "seconds")

* Stage 6: Extracting the model files
Stage 6 completed in 0.0 seconds


In [11]:
# ==============================================
# Loading the box descriptions into a dictionary
# ==============================================
tic = time.perf_counter()
print("* Stage 7: Loading the box descriptions into a dictionary")
import pandas as pd

ID_KEY = 'id'
CLASS_KEY = 'class'
NAME_KEY = 'name'

df = pd.read_csv(BOX_DESCRIPTIONS_FILE, names=[ID_KEY, CLASS_KEY])
category_index = {}
for idx, row in df.iterrows():
    category_index[idx+1] = {ID_KEY: row[ID_KEY], NAME_KEY: row[CLASS_KEY]}
toc = time.perf_counter()
print("Stage 7 completed in", round(toc - tic, 2), "seconds")

* Stage 7: Loading the box descriptions into a dictionary
Stage 7 completed in 0.07 seconds


In [12]:
# ==================================
# Loading the frozen model from file
# ==================================
tf.get_logger().setLevel('WARNING')

@ray.remote
def stage8to10():
    tic = time.perf_counter()
    print("* Stage 8: Loading the frozen model from file")
    graph = tf.Graph()
    with graph.as_default():
        graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
            serialized_graph = fid.read()
            graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(graph_def, name='')
    toc = time.perf_counter()
    print("Stage 8 completed in", round(toc - tic, 2), "seconds")
    
    # =====================
    # Running the inference
    # =====================
    tic = time.perf_counter()
    print("* Stage 9: Running the inference")
    IMAGE_NP_KEY = 'image_np'
    RESULTS_KEY = 'results'
    
    file_result_dict = {}
    
    for filename in TEST_IMAGES:
        image_np = load_image_into_numpy_array(Image.open(filename))
        
        output_dict = run_inference(graph, image_np)
 
        results = process_output(output_dict[DETECTION_CLASSES_KEY],
                                 output_dict[DETECTION_SCORES_KEY],
                                 output_dict[DETECTION_BOXES_KEY],
                                 category_index)

        file_result_dict[filename] = { IMAGE_NP_KEY: image_np, RESULTS_KEY: results }
    
    toc = time.perf_counter()
    print("Stage 9 completed in", round(toc - tic, 2), "seconds")
    
    # ===================
    # Showing the results
    # ===================
    tic = time.perf_counter()
    print("* Stage 10: Showing the results")
    for filename, img_results_dict in file_result_dict.items():
        detected = []
        for r in img_results_dict[RESULTS_KEY]:
            if r.score >= .4:
                 detected.append(str(r))
                 #print(filename, "Identified as", str(r))
        print(filename, "Identified as", ', '.join(detected))
    toc = time.perf_counter()
    print("Stage 10 completed in", round(toc - tic, 2), "seconds")
    
stage8to10.remote()

ObjectRef(ca80f3a8e8ba2e50ffffffffffffffffffffffff0400000001000000)

In [13]:
# Total execution time
end_time = time.perf_counter()
print("Total execution time:", round(end_time - start_time, 2), "seconds")

Total execution time: 4.65 seconds
(stage8to10 pid=800765) * Stage 8: Loading the frozen model from file
(stage8to10 pid=800765) Stage 8 completed in 8.23 seconds
(stage8to10 pid=800765) * Stage 9: Running the inference


(stage8to10 pid=800765) 2022-07-27 02:56:07.210112: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
(stage8to10 pid=800765) 2022-07-27 02:56:07.210146: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
(stage8to10 pid=800765) 2022-07-27 02:56:07.210166: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mgmt01): /proc/driver/nvidia/version does not exist
(stage8to10 pid=800765) 2022-07-27 02:56:07.210769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(stage8to10 pid=80076

(stage8to10 pid=800765) Stage 9 completed in 175.69 seconds
(stage8to10 pid=800765) * Stage 10: Showing the results
(stage8to10 pid=800765) images/dog.jpg Identified as Dog (99%)
(stage8to10 pid=800765) images/girl.jpg Identified as Human face (96%), Girl (90%), Drink (80%), Clothing (78%)
(stage8to10 pid=800765) images/pets.jpg Identified as Dog (99%), Cat (95%)
(stage8to10 pid=800765) images/man.jpg Identified as Human face (93%), Clothing (74%), Man (51%)
(stage8to10 pid=800765) images/man2.jpg Identified as Human face (97%), Clothing (87%), Man (50%), Person (45%)
(stage8to10 pid=800765) images/man3.jpg Identified as Man (74%), Person (44%)
(stage8to10 pid=800765) images/cat.jpg Identified as Cat (99%)
(stage8to10 pid=800765) Stage 10 completed in 0.0 seconds
